In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from scipy.io import loadmat, savemat

In [ ]:
# Define functions
import torch
import numpy as np
import scipy.io
import h5py
import torch.nn as nn

import operator
from functools import reduce
from functools import partial

# Utilities

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# reading data
class MatReader(object):
    def __init__(self, file_path, to_torch=True, to_cuda=False, to_float=True):
        super(MatReader, self).__init__()

        self.to_torch = to_torch
        self.to_cuda = to_cuda
        self.to_float = to_float

        self.file_path = file_path

        self.data = None
        self.old_mat = None
        self._load_file()

    def _load_file(self):
        try:
            self.data = scipy.io.loadmat(self.file_path)
            self.old_mat = True
        except:
            self.data = h5py.File(self.file_path)
            self.old_mat = False

    def load_file(self, file_path):
        self.file_path = file_path
        self._load_file()

    def read_field(self, field):
        x = self.data[field]

        if not self.old_mat:
            x = x[()]
            x = np.transpose(x, axes=range(len(x.shape) - 1, -1, -1))

        if self.to_float:
            x = x.astype(np.float32)

        if self.to_torch:
            x = torch.from_numpy(x)

            if self.to_cuda:
                x = x.cuda()

        return x

    def set_cuda(self, to_cuda):
        self.to_cuda = to_cuda

    def set_torch(self, to_torch):
        self.to_torch = to_torch

    def set_float(self, to_float):
        self.to_float = to_float

# normalization, pointwise gaussian
class UnitGaussianNormalizer(object):
    def __init__(self, x, eps=0.00001):
        super(UnitGaussianNormalizer, self).__init__()

        # x could be in shape of ntrain*n or ntrain*T*n or ntrain*n*T
        self.mean = torch.mean(x, 0)
        self.std = torch.std(x, 0)
        self.eps = eps

    def encode(self, x):
        x = (x - self.mean) / (self.std + self.eps)
        return x

    def decode(self, x, sample_idx=None):
        if sample_idx is None:
            std = self.std + self.eps # n
            mean = self.mean
        else:
            if len(self.mean.shape) == len(sample_idx[0].shape):
                std = self.std[sample_idx] + self.eps  # batch*n
                mean = self.mean[sample_idx]
            if len(self.mean.shape) > len(sample_idx[0].shape):
                std = self.std[:,sample_idx]+ self.eps # T*batch*n
                mean = self.mean[:,sample_idx]

        # x is in shape of batch*n or T*batch*n
        x = (x * std) + mean
        return x

    def cuda(self):
        self.mean = self.mean.cuda()
        self.std = self.std.cuda()

    def cpu(self):
        self.mean = self.mean.cpu()
        self.std = self.std.cpu()

# normalization, Gaussian
class GaussianNormalizer(object):
    def __init__(self, x, eps=0.00001):
        super(GaussianNormalizer, self).__init__()

        self.mean = torch.mean(x)
        self.std = torch.std(x)
        self.eps = eps

    def encode(self, x):
        x = (x - self.mean) / (self.std + self.eps)
        return x

    def decode(self, x, sample_idx=None):
        x = (x * (self.std + self.eps)) + self.mean
        return x

    def cuda(self):
        self.mean = self.mean.cuda()
        self.std = self.std.cuda()

    def cpu(self):
        self.mean = self.mean.cpu()
        self.std = self.std.cpu()


# normalization, scaling by range
class RangeNormalizer(object):
    def __init__(self, x, low=0.0, high=1.0):
        super(RangeNormalizer, self).__init__()
        mymin = torch.min(x, 0)[0].view(-1)
        mymax = torch.max(x, 0)[0].view(-1)

        self.a = (high - low)/(mymax - mymin)
        self.b = -self.a*mymax + high

    def encode(self, x):
        s = x.size()
        x = x.view(s[0], -1)
        x = self.a*x + self.b
        x = x.view(s)
        return x

    def decode(self, x):
        s = x.size()
        x = x.view(s[0], -1)
        x = (x - self.b)/self.a
        x = x.view(s)
        return x

#loss function with rel/abs Lp loss
class LpLoss(object):
    def __init__(self, d=2, p=2, size_average=True, reduction=True):
        super(LpLoss, self).__init__()

        #Dimension and Lp-norm type are postive
        assert d > 0 and p > 0

        self.d = d
        self.p = p
        self.reduction = reduction
        self.size_average = size_average

    def abs(self, x, y):
        num_examples = x.size()[0]

        #Assume uniform mesh
        h = 1.0 / (x.size()[1] - 1.0)

        all_norms = (h**(self.d/self.p))*torch.norm(x.view(num_examples,-1) - y.view(num_examples,-1), self.p, 1)

        if self.reduction:
            if self.size_average:
                return torch.mean(all_norms)
            else:
                return torch.sum(all_norms)

        return all_norms

    def rel(self, x, y):
        num_examples = x.size()[0]

        diff_norms = torch.norm(x.reshape(num_examples,-1) - y.reshape(num_examples,-1), self.p, 1)
        y_norms = torch.norm(y.reshape(num_examples,-1), self.p, 1)

        if self.reduction:
            if self.size_average:
                return torch.mean(diff_norms/y_norms)
            else:
                return torch.sum(diff_norms/y_norms)

        return diff_norms/y_norms

    def __call__(self, x, y):
        return self.rel(x, y)

# Sobolev norm (HS norm)
# where we also compare the numerical derivatives between the output and target
class HsLoss(object):
    def __init__(self, d=2, p=2, k=1, a=None, group=False, size_average=True, reduction=True):
        super(HsLoss, self).__init__()

        #Dimension and Lp-norm type are postive
        assert d > 0 and p > 0

        self.d = d
        self.p = p
        self.k = k
        self.balanced = group
        self.reduction = reduction
        self.size_average = size_average

        if a == None:
            a = [1,] * k
        self.a = a

    def rel(self, x, y):
        num_examples = x.size()[0]
        diff_norms = torch.norm(x.reshape(num_examples,-1) - y.reshape(num_examples,-1), self.p, 1)
        y_norms = torch.norm(y.reshape(num_examples,-1), self.p, 1)
        if self.reduction:
            if self.size_average:
                return torch.mean(diff_norms/y_norms)
            else:
                return torch.sum(diff_norms/y_norms)
        return diff_norms/y_norms

    def __call__(self, x, y, a=None):
        nx = x.size()[1]
        ny = x.size()[2]
        k = self.k
        balanced = self.balanced
        a = self.a
        x = x.view(x.shape[0], nx, ny, -1)
        y = y.view(y.shape[0], nx, ny, -1)

        k_x = torch.cat((torch.arange(start=0, end=nx//2, step=1),torch.arange(start=-nx//2, end=0, step=1)), 0).reshape(nx,1).repeat(1,ny)
        k_y = torch.cat((torch.arange(start=0, end=ny//2, step=1),torch.arange(start=-ny//2, end=0, step=1)), 0).reshape(1,ny).repeat(nx,1)
        k_x = torch.abs(k_x).reshape(1,nx,ny,1).to(x.device)
        k_y = torch.abs(k_y).reshape(1,nx,ny,1).to(x.device)

        x = torch.fft.fftn(x, dim=[1, 2])
        y = torch.fft.fftn(y, dim=[1, 2])

        if balanced==False:
            weight = 1
            if k >= 1:
                weight += a[0]**2 * (k_x**2 + k_y**2)
            if k >= 2:
                weight += a[1]**2 * (k_x**4 + 2*k_x**2*k_y**2 + k_y**4)
            weight = torch.sqrt(weight)
            loss = self.rel(x*weight, y*weight)
        else:
            loss = self.rel(x, y)
            if k >= 1:
                weight = a[0] * torch.sqrt(k_x**2 + k_y**2)
                loss += self.rel(x*weight, y*weight)
            if k >= 2:
                weight = a[1] * torch.sqrt(k_x**4 + 2*k_x**2*k_y**2 + k_y**4)
                loss += self.rel(x*weight, y*weight)
            loss = loss / (k+1)

        return loss

# A simple feedforward neural network
class DenseNet(torch.nn.Module):
    def __init__(self, layers, nonlinearity, out_nonlinearity=None, normalize=False):
        super(DenseNet, self).__init__()

        self.n_layers = len(layers) - 1

        assert self.n_layers >= 1

        self.layers = nn.ModuleList()

        for j in range(self.n_layers):
            self.layers.append(nn.Linear(layers[j], layers[j+1]))

            if j != self.n_layers - 1:
                if normalize:
                    self.layers.append(nn.BatchNorm1d(layers[j+1]))

                self.layers.append(nonlinearity())

        if out_nonlinearity is not None:
            self.layers.append(out_nonlinearity())

    def forward(self, x):
        for _, l in enumerate(self.layers):
            x = l(x)

        return x


# print the number of parameters
def count_params(model):
    c = 0
    for p in list(model.parameters()):
        c += reduce(operator.mul, list(p.size()))
    return c


## 1.2. FNO

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import matplotlib.pyplot as plt

import operator
from functools import reduce
from functools import partial
from timeit import default_timer

torch.manual_seed(0)
np.random.seed(0)


#  1d fourier layer

class SpectralConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1):
        super(SpectralConv1d, self).__init__()

        """
        1D Fourier layer. It does FFT, linear transform, and Inverse FFT.    
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1  #Number of Fourier modes to multiply, at most floor(N/2) + 1

        self.scale = (1 / (in_channels*out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul1d(self, input, weights):
        # (batch, in_channel, x ), (in_channel, out_channel, x) -> (batch, out_channel, x)
        return torch.einsum("bix,iox->box", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        #Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = torch.fft.rfft(x)

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels, x.size(-1)//2 + 1,  device=x.device, dtype=torch.cfloat)
        out_ft[:, :, :self.modes1] = self.compl_mul1d(x_ft[:, :, :self.modes1], self.weights1)

        #Return to physical space
        x = torch.fft.irfft(out_ft, n=x.size(-1))
        return x

class FNO1d(nn.Module):
    def __init__(self, modes, width):
        super(FNO1d, self).__init__()

        """
        The overall network. It contains 4 layers of the Fourier layer.
        1. Lift the input to the desire channel dimension by self.fc0 .
        2. 4 layers of the integral operators u' = (W + K)(u).
            W defined by self.w; K defined by self.conv .
        3. Project from the channel space to the output space by self.fc1 and self.fc2 .
        
        input: the solution of the initial condition and location (a(x), x)
        input shape: (batchsize, x=s, c=2)
        output: the solution of a later timestep
        output shape: (batchsize, x=s, c=1)
        """

        self.modes1 = modes
        self.width = width
        self.fc0 = nn.Linear(2, self.width) # input channel is 2: (a(x), x)

        self.conv0 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv1 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv2 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv3 = SpectralConv1d(self.width, self.width, self.modes1)
        self.w0 = nn.Conv1d(self.width, self.width, 1)
        self.w1 = nn.Conv1d(self.width, self.width, 1)
        self.w2 = nn.Conv1d(self.width, self.width, 1)
        self.w3 = nn.Conv1d(self.width, self.width, 1)


        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):

        x = self.fc0(x)
        x = x.permute(0, 2, 1)

        x1 = self.conv0(x)
        x2 = self.w0(x)
        x = x1 + x2
        x = F.relu(x)

        x1 = self.conv1(x)
        x2 = self.w1(x)
        x = x1 + x2
        x = F.relu(x)

        x1 = self.conv2(x)
        x2 = self.w2(x)
        x = x1 + x2
        x = F.relu(x)

        x1 = self.conv3(x)
        x2 = self.w3(x)
        x = x1 + x2

        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
# Coupled Data

ntrain = 1000
ntest = 200

sub = 2**0 #subsampling rate
h = 2**8 // sub #total grid size divided by the subsampling rate
s = h


batch_size = 20

rw_u = loadmat('/content/drive/MyDrive/gray_scott_results/Coupled_PDE_data/kernel1Drho_t0_1.mat')
x_data = rw_u['rho_t0'].astype(np.float32)
y_data = rw_u['rho_t02'].astype(np.float32)
print(x_data.shape)

x_train_u = x_data[:ntrain,::sub]
y_train_u = y_data[:ntrain,::sub]
x_test_u = x_data[-ntest:,::sub]
y_test_u = y_data[-ntest:,::sub]

x_train_u = torch.from_numpy(x_train_u)
x_test_u = torch.from_numpy(x_test_u)
y_train_u = torch.from_numpy(y_train_u)
y_test_u = torch.from_numpy(y_test_u)

x_train_u = x_train_u.unsqueeze(-1)
x_test_u = x_test_u.unsqueeze(-1)
grid = np.linspace(0, 1, s).reshape(1, s, 1)
grid = torch.tensor(grid, dtype=torch.float)

x_train_u = torch.cat([x_train_u.reshape(ntrain,s,1), grid.repeat(ntrain,1,1)], dim=2)
x_test_u = torch.cat([x_test_u.reshape(ntest,s,1), grid.repeat(ntest,1,1)], dim=2)


rw_v = loadmat('/content/drive/MyDrive/gray_scott_results/Coupled_PDE_data/kernel1Dphi_t0_1.mat')
x_data = rw_v['phi_t0'].astype(np.float32)
y_data = rw_v['phi_t02'].astype(np.float32)

x_train_v = x_data[:ntrain,::sub]
y_train_v = y_data[:ntrain,::sub]
x_test_v = x_data[-ntest:,::sub]
y_test_v = y_data[-ntest:,::sub]

x_train_v = torch.from_numpy(x_train_v)
x_test_v = torch.from_numpy(x_test_v)
y_train_v = torch.from_numpy(y_train_v)
y_test_v = torch.from_numpy(y_test_v)
print(y_test_u.shape)

x_train_v = x_train_v.unsqueeze(-1)
x_test_v = x_test_v.unsqueeze(-1)

x_train_v = torch.cat([x_train_v.reshape(ntrain,s,1), grid.repeat(ntrain,1,1)], dim=2)
x_test_v = torch.cat([x_test_v.reshape(ntest,s,1), grid.repeat(ntest,1,1)], dim=2)


x_train = torch.cat([x_train_u.reshape(ntrain,s,-1), x_train_v.reshape(ntrain,s,-1)], dim=1)
x_test = torch.cat([x_test_u.reshape(ntest,s,-1), x_test_v.reshape(ntest,s,-1)], dim=1)

y_train = torch.cat([y_train_u.reshape(ntrain,s,-1), y_train_v.reshape(ntrain,s,-1)], dim=1)
y_test = torch.cat([y_test_u.reshape(ntest,s,-1), y_test_v.reshape(ntest,s,-1)], dim=1)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

(1200, 1024)
torch.Size([200, 1024])


In [ ]:
# model
learning_rate = 0.001

epochs = 500
step_size = 100
gamma = 0.5

modes = 16
width = 64

model = FNO1d(modes, width).cuda()
print(count_params(model))


287425


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

maeloss = nn.L1Loss()
myloss = LpLoss(size_average=False)

error_u = []
error_v = []

mae_error_u = []
mae_error_v = []

for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_mse = 0
    train_l2 = 0
    for x, y in train_loader:
        x, y = x.cuda(), y.cuda()

        optimizer.zero_grad()
        out = model(x)
        mse = F.mse_loss(out.view(batch_size, -1), y.view(batch_size, -1), reduction='mean')
        # mse.backward()
        l2 = myloss(out.view(batch_size, -1), y.view(batch_size, -1))
        l2.backward() # use the l2 relative loss

        optimizer.step()
        train_mse += mse.item()
        train_l2 += l2.item()

    scheduler.step()
    model.eval()
    test_l2 = 0.0
    test_l2_u = 0.0
    test_l2_v = 0.0
    test_mae = 0.0
    test_mae_u = 0.0
    test_mae_v = 0.0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.cuda(), y.cuda()

            out = model(x)
            test_l2 += myloss(out.view(batch_size, -1), y.view(batch_size, -1)).item()
            test_l2_u += myloss(out[:,:s,:].view(batch_size, -1), y[:,:s,:].view(batch_size, -1)).item()
            test_l2_v += myloss(out[:,s:,:].view(batch_size, -1), y[:,s:,:].view(batch_size, -1)).item()
            test_mae += maeloss(out.view(batch_size, -1), y.view(batch_size, -1)).item()
            test_mae_u += maeloss(out[:,:s,:].view(batch_size, -1), y[:,:s,:].view(batch_size, -1)).item()
            test_mae_v += maeloss(out[:,s:,:].view(batch_size, -1), y[:,s:,:].view(batch_size, -1)).item()           

    train_mse /= len(train_loader)
    train_l2 /= ntrain
    test_l2 /= ntest
    test_l2_u /= ntest
    test_l2_v /= ntest
    test_mae /= ntest
    test_mae_u /= ntest
    test_mae_v /= ntest
    error_u.append(test_l2_u)
    error_v.append(test_l2_v)
    mae_error_u.append(test_mae_u)
    mae_error_v.append(test_mae_v)

    t2 = default_timer()
    print(ep, t2-t1, train_mse,test_mae, test_mae_u,test_mae_v,train_l2, test_l2, test_l2_u, test_l2_v)

# np.save('/content/drive/MyDrive/gray_scott_results/Gray_scott_u_l02l02_ugrf_vgrf_FNO_cat_error',error_u)
# np.save('/content/drive/MyDrive/gray_scott_results/Gray_scott_v_l02l02_ugrf_vgrf_FNO_cat_error',error_v)
# np.save('/content/drive/MyDrive/gray_scott_results/Gray_scott_u_l02l02_ugrf_vgrf_FNO_cat_mae',mae_error_u)
# np.save('/content/drive/MyDrive/gray_scott_results/Gray_scott_v_l02l02_ugrf_vgrf_FNO_cat_mae',mae_error_v)

0 0.8966476069999771 0.012514422265812754 0.002357453368604183 0.00233908711001277 0.002375819571316242 0.4741417555809021 0.3059979724884033 0.5252103185653687 0.2668923544883728
1 0.8482604640000204 0.0035703663481399415 0.002281180117279291 0.002068302873522043 0.002494057435542345 0.2938236141204834 0.2953681516647339 0.46636282444000243 0.27578905582427976
2 0.8498267580000061 0.0034674059692770243 0.002276374623179436 0.0021233640797436236 0.0024293852038681507 0.288896324634552 0.29174174785614015 0.4790955996513367 0.26469601392745973
3 0.8462420500000007 0.003379248445853591 0.0022398486360907556 0.0020893441513180734 0.0023903531022369862 0.28563507986068726 0.28836311101913453 0.4731708431243897 0.25903055429458616
4 0.8472944220000045 0.003377779186703265 0.002255691010504961 0.0019164248183369636 0.0025949571281671524 0.2851783604621887 0.29132478952407836 0.43755842447280885 0.27923821926116943
5 0.8555098040000075 0.003311974979005754 0.0022367834486067295 0.001996588427